In [1]:
import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow
from sklearn.metrics import (accuracy_score, confusion_matrix, f1_score,
                             precision_score, recall_score)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

%matplotlib inline

Using TensorFlow backend.


【問題1】SimpleRNNのフォワードプロパゲーション実装
SimpleRNNのクラスSimpleRNNを作成してください。基本構造はFCクラスと同じになります。

フォワードプロパゲーションの数式は以下のようになります。ndarrayのshapeがどうなるかを併記しています。

バッチサイズをbatch_size、入力の特徴量数をn_features、RNNのノード数をn_nodesとして表記します。活性化関数はtanhとして進めますが、これまでのニューラルネットワーク同様にReLUなどに置き換えられます。

a
t
=
x
t
⋅
W
x
+
h
t
−
1
⋅
W
h
+
B
h
t
=
t
a
n
h
(
a
t
)
a
t
 : 時刻tの活性化関数を通す前の状態 (batch_size, n_nodes)

h
t
 : 時刻tの状態・出力 (batch_size, n_nodes)

x
t
 : 時刻tの入力 (batch_size, n_features)

W
x
 : 入力に対する重み (n_features, n_nodes)

h
t
−
1
 : 時刻t-1の状態（前の時刻から伝わる順伝播） (batch_size, n_nodes)

W
h
 : 状態に対する重み。 (n_nodes, n_nodes)

B
 : バイアス項 (n_nodes,)

初期状態 
h
0
 は全て0とすることが多いですが、任意の値を与えることも可能です。

上記の処理を系列数n_sequences回繰り返すことになります。RNN全体への入力 
x
 は(batch_size, n_sequences, n_features)のような配列で渡されることになり、そこから各時刻の配列を取り出していきます。

分類問題であれば、それぞれの時刻のhに対して全結合層とソフトマックス関数（またはシグモイド関数）を使用します。タスクによっては最後の時刻のhだけを使用することもあります。

In [2]:
# x = np.array([[[1, 2], [2, 3], [3, 4]]])/100 # (batch_size, n_sequences, n_features)
# w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100 # (n_features, n_nodes)
# w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100 # (n_nodes, n_nodes)
# batch_size = x.shape[0] # 1
# n_sequences = x.shape[1] # 3
# n_features = x.shape[2] # 2
# n_nodes = w_x.shape[1] # 4
# h = np.zeros((batch_size, n_nodes)) # (batch_size, n_nodes)
# b = np.array([1, 1, 1, 1]) # (n_nodes,)

In [3]:
# #batch数分ループ
# for i in range(n_sequences):
#     a_t = x[:, i, :]@w_x + h@w_h + b
#     h = np.tanh(a_t)
# print(h)

In [4]:
class SimpleRNN:
    """
    ノード数n_nodes1からn_nodes2への全結合層
    Parameters
    ----------
    n_nodes : int
      前の層のノード数
    """
    def __init__(self):
        self.w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100 # (n_features, n_nodes)
        self.w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100 # (n_nodes, n_nodes)
        self.b = np.array([1, 1, 1, 1]) # (n_nodes,)
        self.n_nodes = self.w_x.shape[1] # 4
        pass
    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_sequences, n_features)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes)
            出力
        """ 
        batch_size = X.shape[0] # 1
        n_sequences = X.shape[1] # 3
        n_features = X.shape[2] # 2

        h = np.zeros((batch_size, self.n_nodes))

        
        for i in range(n_sequences):
            a_t = X[:, i, :]@self.w_x + h@self.w_h + self.b
            h = np.tanh(a_t)
            
        return h
    
    def tanh(self, X): # ハイパボリックタンジェント関数

        #(np.exp(X)-np.exp(-X)) / (np.exp(X) + np.exp(-X))

        return np.tanh(X)

    def backward(self, dA):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """
        pass
        # 更新
        self = self.optimizer.update(self)
        return dZ

In [5]:
#インスタンス化
Srnn = SimpleRNN()

In [6]:
#フォーワードプロパゲーション
x = np.array([[[1, 2], [2, 3], [3, 4]]])/100
Srnn.forward(x)

array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]])